In [8]:
import requests
import pandas as pd
from io import StringIO

I will download the files of each broker and make it in proper format for further processing

### Angel One

The angelone broking scriptmaster is present on the given url in json format. We will download the data and convert it to csv format

In [2]:
url = 'https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json'

In [3]:
response = requests.get(url)
data = response.json() 

In [4]:
df = pd.json_normalize(data)


In [6]:
csv_file_path = "D:\\stock\\angelone.csv"
df.to_csv(csv_file_path, index=False)

In [7]:
df.head()

,token,symbol,name,expiry,strike,lotsize,instrumenttype,exch_seg,tick_size
0,6293,GULPOLY-BL,GULPOLY,,-1.000000,1,,NSE,1.000000
1,24051,739UK30-SG,739UK30,,-1.000000,100,,NSE,1.000000
2,3475,THERMAX-EQ,THERMAX,,-1.000000,1,,NSE,5.000000
3,3523,69MZ33-SG,69MZ33,,-1.000000,100,,NSE,1.000000
4,17996,PGIMCSR2G-MF,PGIMCSR2G,,-1.000000,1,,NSE,1.000000


### Kotak

There are 2 text url for kotak: Cash & Futures and Options
We will download and merge this file into csv file

In [9]:
url = 'https://preferred.kotaksecurities.com/security/production/TradeApiInstruments_Cash_01_04_2022.txt'


In [10]:
response = requests.get(url)
data = response.text

In [11]:
df = pd.read_csv(StringIO(data), sep='|')


In [12]:
df= df.rename(columns={'OptionType': 'optionType'})


In [13]:
df.head()

,instrumentToken,instrumentName,name,lastPrice,expiry,strike,tickSize,lotSize,instrumentType,segment,exchange,isin,multiplier,exchangeToken,optionType
0,9516,20MICRONS,20 Microns Ltd,73.85,0,0,0.05,1,EQ,CASH,NSE,INE144J01027,1,16921,-
1,43865,3IINFOLTD,3i Infotech Ltd,51.25,0,0,0.05,1,EQ,CASH,NSE,INE748C01038,1,6232,-
2,891,3MINDIA,3M India Ltd,19697.30,0,0,0.05,1,EQ,CASH,NSE,INE470A01017,1,474,-
3,607,3PLAND,3P Land Holdings Ltd,14.50,0,0,0.05,1,EQ,CASH,NSE,INE105C01023,1,2595,-
4,13140,5PAISA,5Paisa Capital Ltd,344.70,0,0,0.05,1,EQ,CASH,NSE,INE618L01018,1,445,-


In [14]:
csv_file_path = "D:\\stock\\kotak-cash.csv"
df.to_csv(csv_file_path, index=False)

In [15]:
url="https://preferred.kotaksecurities.com/security/production/TradeApiInstruments_FNO_01_04_2022.txt"

In [16]:
response = requests.get(url)
data = response.text

In [17]:
df2 = pd.read_csv(StringIO(data), sep='|')


In [18]:
csv_file_path = "D:\\stock\\kotak-FNO.csv"
df2.to_csv(csv_file_path, index=False)

In [19]:
combined_df = pd.concat([df, df2], ignore_index=True)


In [20]:
combined_df.to_csv("D:\\stock\\kotak.csv",index=False)

### Fyers 

There are 6 urls for fyers data. 
We get this data from given url:https://community.fyers.in/questions-5gz5j8db/post/symbol-master-file-header-correction-QjPqAvb6bTaaSgU

The headers in the given file are missing. But we get from the given link: https://myapi.fyers.in/docsv3#tag/Broker-Config/paths/~1Broker%20Config/patch

In [21]:
# Define URLs for each CSV file
urls = {
    "NSE_CD.csv": "https://public.fyers.in/sym_details/NSE_CD.csv",
    "NSE_FO.csv": "https://public.fyers.in/sym_details/NSE_FO.csv",
    "NSE_CM.csv": "https://public.fyers.in/sym_details/NSE_CM.csv",
    "BSE_CM.csv": "https://public.fyers.in/sym_details/BSE_CM.csv",
    "BSE_FO.csv": "https://public.fyers.in/sym_details/BSE_FO.csv",
    "MCX_COM.csv": "https://public.fyers.in/sym_details/MCX_COM.csv"
}

In [22]:
# Define headers for each file
headers = [
    "Fytoken", "Symbol Details", "Exchange Instrument type", "Minimum lot size",
    "Tick size", "ISIN", "Trading Session", "Last update date", "Expiry date",
    "Symbol ticker", "Exchange", "Segment", "Scrip code", "Underlying symbol",
    "Underlying scrip code", "Strike price", "Option type", "Underlying FyToken",
    "Reserved column", "Reserved column", "Reserved column"
]

In [23]:
dataframes = []


In [24]:
# Loop over each file, download it, add headers, and append to the list
for url in urls.values():
    # Download the CSV file
    response = requests.get(url)
    data = response.content.decode('utf-8')
    
    # Read the CSV data into a DataFrame, with no header
    df = pd.read_csv(StringIO(data), header=None)
    
    # Add headers to the DataFrame
    df.columns = headers
    
    # Append the DataFrame to the list
    dataframes.append(df)

In [25]:
combined_df = pd.concat(dataframes, ignore_index=True)


In [26]:
output_file_path = "D:\\stock\\fyers.csv"
combined_df.to_csv(output_file_path, index=False)


In [27]:
combined_df.head()

,Fytoken,Symbol Details,Exchange Instrument type,Minimum lot size,Tick size,ISIN,Trading Session,Last update date,Expiry date,Symbol ticker,...,Segment,Scrip code,Underlying symbol,Underlying scrip code,Strike price,Option type,Underlying FyToken,Reserved column,Reserved column,Reserved column
0,101224102910107,JPYINR 24 Oct 29 65.5 CE,19,1,0.0025,NaN,0900-1700|1815-1915:,2024-10-25,1730185200,NSE:JPYINR24OCT65.5CE,...,12,10107,JPYINR,27,65.500,CE,101200000027,None,0,0.0
1,101224102910109,JPYINR 24 Oct 29 65.5 PE,19,1,0.0025,NaN,0900-1700|1815-1915:,2024-10-25,1730185200,NSE:JPYINR24OCT65.5PE,...,12,10109,JPYINR,27,65.500,PE,101200000027,None,0,0.0
2,10122410291032,EURINR 24 Oct 29 84.75 CE,19,1,0.0025,NaN,0900-1700|1815-1915:,2024-10-25,1730185200,NSE:EURINR24OCT84.75CE,...,12,1032,EURINR,25,84.750,CE,101200000025,None,0,0.0
3,10122410291033,EURUSD 24 Oct 29 1.225 CE,19,1,0.0001,NaN,0900-1700|1815-1915:,2024-10-25,1730185200,NSE:EURUSD24OCT1.225CE,...,12,1033,EURUSD,41,1.225,CE,101200000041,None,0,0.0
4,10122410291034,EURUSD 24 Oct 29 1.225 PE,19,1,0.0001,NaN,0900-1700|1815-1915:,2024-10-25,1730185200,NSE:EURUSD24OCT1.225PE,...,12,1034,EURUSD,41,1.225,PE,101200000041,None,0,0.0


### Zerodha

We get the zerodha instrument list from the given url: https://api.kite.trade/instruments

In [28]:
z=pd.read_csv("D:\\stock\\zerodha.csv")

In [29]:
z.head()

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,264741126,1034145,EURINR24OCTFUT,EURINR,0,2024-10-29,0.0,0.0025,1,FUT,BCD-FUT,BCD
1,265969926,1038945,EURINR24NOVFUT,EURINR,0,2024-11-27,0.0,0.0025,1,FUT,BCD-FUT,BCD
2,268449030,1048629,EURINR24DECFUT,EURINR,0,2024-12-27,0.0,0.0025,1,FUT,BCD-FUT,BCD
3,269248262,1051751,EURINR25JANFUT,EURINR,0,2025-01-29,0.0,0.0025,1,FUT,BCD-FUT,BCD
4,270018310,1054759,EURINR25FEBFUT,EURINR,0,2025-02-26,0.0,0.0025,1,FUT,BCD-FUT,BCD
